This is the setup cell

In [2]:
import datetime
import pathlib
import time
import math

import obspy
from obspy.signal import PPSD
import scipy
import matplotlib.pyplot as plt
import matplotlib
#Run this line if you want interactive plots
%matplotlib qt
import numpy as np

import sprit

In [2]:
import matplotlib
matplotlib.rcsetup.interactive_bk

['GTK3Agg',
 'GTK3Cairo',
 'GTK4Agg',
 'GTK4Cairo',
 'MacOSX',
 'nbAgg',
 'QtAgg',
 'QtCairo',
 'Qt5Agg',
 'Qt5Cairo',
 'TkAgg',
 'TkCairo',
 'WebAgg',
 'WX',
 'WXAgg',
 'WXCairo']

In [3]:
dPath = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake'
#dPath = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake\Test12_AM.RAC84.00.2023-02-15_2132-2200.mseed'
trimDir = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake'
#dPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\ChampaignCo\Mahomet2023\Data\RawData"
dPath = r"C:\Users\riley\OneDrive - University of Illinois - Urbana\Data_OneDrive\Seismic\HVSR\Data\RAC84"
dPath = r"C:\Users\riley\OneDrive - University of Illinois - Urbana\Data_OneDrive\Seismic\HVSR\Data\UOFI1_AM.RAC84.00.2023-02-15_1704-1734.mseed"

params = sprit.input_param( acq_date='2023-02-06',
                            #starttime = obspy.UTCDateTime('2023-02-06T21:32:00.00'),
                            #endtime = obspy.UTCDateTime('2023-02-06T22:00:00.00'),
                            tzone = 'UTC',#or 'local', #or 'UTC'
                            dst=False,
                            lon = -88.2290526,
                            lat =  40.1012122,
                            elevation = 755,
                            site = 'HVSR Site',
                            dataPath = dPath
                            )

No metadata file specified!
Using default metadata file for Raspberry Shake v.7 contained in repository at
 c:/Users/riley/LocalData/Github/SPRIT/resources/raspshake_metadata.inv


In [4]:
params = sprit.get_metadata(params)
params = sprit.fetch_data(params=params, source='raw')

Day of Year: 37


c:\Users\riley\anaconda3\envs\seismic39\lib\site-packages\obspy\core\inventory\network.py:251: UserWarning: Found more than one matching response. Returning first.
  warnings.warn(msg)


In [5]:
params = sprit.remove_noise(input=params, kind='manual')
params = sprit.remove_noise(input=params, kind='auto', noise_percent = 0.9, show_windows=False)

c:\Users\riley\anaconda3\envs\seismic39\lib\site-packages\obspy\signal\trigger.py:183: UserWarning: Warning: converting a masked element to nan.
  data = np.ascontiguousarray(data, dtype=np.float64)


In [6]:
params = sprit.generate_ppsds(params=params, stream=params['stream'], ppsd_length=60, 
                              overlap=0.5, period_step_octaves=0.01, remove_outliers=True, outlier_std=1.5)

In [7]:
hvsr_dict = sprit.process_hvsr(params=params, method=4, resample=True, smooth=3, freq_smooth=False, f_smooth_width=10, remove_outlier_curves=True)
hvsr_dict = sprit.check_peaks(hvsr_dict=hvsr_dict)

No frequency smoothing is being applied. This is not recommended for noisy datasets.


In [8]:
sprit.hvplot(hvsr_dict, kind='hvsr p t ann spec', cmap='turbo')